In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model, layers
from tensorflow import keras

import numpy as np

In [22]:
def generate_data():
    X = np.random.randint(0, 10, (10000,30))
    X = X[:, :, np.newaxis]
#     X = X.astype('float32')
    YT = X.sum(1) >= 100
#     YT = YT.astype('float32')
    
    X_train = X[:8000]
    X_test = X[8000:]
    YT_train = YT[:8000]
    YT_test = YT[8000:]
    
    return X_train, YT_train, X_test, YT_test

X_train, YT_train, X_test, YT_test = generate_data()
YT_train.dtype

dtype('bool')

In [23]:
print((X_train.sum(1) >= 100).sum()/len(X_train) * 100, "% of training set is equal or over 100")

98.875 % of training set is equal or over 100


In [24]:
# class variant
class MyModel(Sequential):
    def __init__(self):
        super(MyModel, self).__init__()
        
    def summary(self, input_shape=(30,1)):
        x = layers.Input(shape=input_shape)
        model = tf.keras.Model(inputs=x, outputs=self.call(x))
        print(model.summary())
        
class LSTM(MyModel):
    def __init__(self):
        super(LSTM, self).__init__()
    
    def call(self, x):
        x = layers.LSTM(
            units=200,
        )(x)
        x = layers.Dense(
            1,
            activation='sigmoid'
        )(x)
        return x

model = LSTM()
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 30, 1)]           0         
                                                                 
 lstm_10 (LSTM)              (None, 200)               161600    
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 161,801
Trainable params: 161,801
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
# not class variant
model = Sequential()
model.add(layers.LSTM(200, input_shape=(30,1)))
model.add(layers.Dense(1, activation='sigmoid'))

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 200)               161600    
                                                                 
 dense_6 (Dense)             (None, 1)                 201       
                                                                 
Total params: 161,801
Trainable params: 161,801
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=keras.metrics.BinaryAccuracy()
)

model.fit(
    X_train, YT_train,
    epochs=60,
    batch_size=50
)

Epoch 1/60
160/160 [==============================] - 4s 21ms/step - loss: 0.0603 - binary_accuracy: 0.9884
Epoch 2/60
160/160 [==============================] - 3s 21ms/step - loss: 0.0501 - binary_accuracy: 0.9885
Epoch 3/60
160/160 [==============================] - 3s 21ms/step - loss: 0.0514 - binary_accuracy: 0.9887
Epoch 4/60
 64/160 [===========>..................] - ETA: 1s - loss: 0.0427 - binary_accuracy: 0.9869

KeyboardInterrupt: 